# Basic checks of CREDIT ERA5 inputs

In [1]:
import numpy as np
import xarray as xr

from glob import glob

## variable names

In [2]:
filenames = sorted(glob('/glade/derecho/scratch/wchapman/SixHourly_y_TOTAL*'))

In [3]:
for i_fn, fn in enumerate(filenames):
    try:
        ds_temp = xr.open_zarr(fn)
        print(list(ds_temp.keys()))
    except:
        print(fn)

['LSM', 'Q', 'Q500', 'SP', 'T', 'T500', 'U', 'U500', 'V', 'V500', 'Z500', 'Z_GDS4_SFC', 't2m', 'tsi']
['Q', 'Q500', 'SP', 'T', 'T500', 'U', 'U500', 'V', 'V500', 'Z500', 't2m']
['Q', 'Q500', 'SP', 'T', 'T500', 'U', 'U500', 'V', 'V500', 'Z500', 't2m']
['Q', 'Q500', 'SP', 'T', 'T500', 'U', 'U500', 'V', 'V500', 'Z500', 't2m']
['Q', 'Q500', 'SP', 'T', 'T500', 'U', 'U500', 'V', 'V500', 'Z500', 't2m']
['Q', 'Q500', 'SP', 'T', 'T500', 'U', 'U500', 'V', 'V500', 'Z500', 't2m']
['Q', 'Q500', 'SP', 'T', 'T500', 'U', 'U500', 'V', 'V500', 'Z500', 't2m']
['Q', 'Q500', 'SP', 'T', 'T500', 'U', 'U500', 'V', 'V500', 'Z500', 't2m']
['Q', 'Q500', 'SP', 'T', 'T500', 'U', 'U500', 'V', 'V500', 'Z500', 't2m']
['Q', 'Q500', 'SP', 'T', 'T500', 'U', 'U500', 'V', 'V500', 'Z500', 't2m']
['Q', 'Q500', 'SP', 'T', 'T500', 'U', 'U500', 'V', 'V500', 'Z500', 't2m']
['Q', 'Q500', 'SP', 'T', 'T500', 'U', 'U500', 'V', 'V500', 'Z500', 't2m']
['Q', 'Q500', 'SP', 'T', 'T500', 'U', 'U500', 'V', 'V500', 'Z500', 't2m']
['Q', 'Q50

## check NaNs

In [6]:
def check_nans_ds(ds):
    for varname in ds.data_vars:
        if ds[varname].isnull().any().compute():
            return True
    return False

In [7]:
filenames = sorted(glob('/glade/derecho/scratch/wchapman/SixHourly_y_TOTAL*'))

for i_fn, fn in enumerate(filenames[::-1]):
    ds_temp = xr.open_zarr(fn)
    has_nans = check_nans_ds(ds_temp)
    print('Dataset contains NaNs: {}'.format(has_nans))
    if has_nans:
        print(fn)

Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: True
/glade/derecho/scratch/wchapman/SixHourly_y_TOTAL_1998-01-01_1998-12-31_staged.zarr
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains NaNs: False
Dataset contains N